<a href="https://colab.research.google.com/github/muratfirat78/CPP_Datasets/blob/main/DUP_Lecture2_Dataquality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Understanding and Preparation

    
### Lecture 2: Data and Its Quality
   
__Part 1: Data Preprocessing dashboard:__ <br>
<ol>
  <li>Data reading: Feature names and types, size  </li>
  <li>Handling missing data: Revoming or filling</li>
  <li>Feature Scaling: Standardization and Normalization  </li>
  <li>Outlier detection/removal</li>
  <li>Handling Imbalanced Data</li>
</ol>

__Part 2:__  <br>
<ol>
  <li>Data Integration and Aggragation</li>   

</ol>

## PART 1: Data Processing dashboard


In [2]:
##### import ipywidgets as widgets
from IPython.display import clear_output
from IPython import display
from ipywidgets import *
from datetime import timedelta,date
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import seaborn as sns
import os
from pathlib import Path


warnings.filterwarnings("ignore")


def ActivateTab(change):

    global wslay,ftlay,features
    global ftlay2,features2,tablayout,tab_1,dtlay,fthboxlay
    global rowheight,dtlay,dtypes,missing,msslay,misshands,mhndslay,curr_df




    if change['new'] == 1:
        ActivateTab1()
    if change['new'] == 2:
        ActivateTab2()



    return


##################################################################################################################
###############################################  TAB: Data Sets ##################################################
##################################################################################################################


def FileClick(change):

    global seplay,septxt,datasets,wslay,wsheets,DataFolder,butlay


    rel_path = DataFolder.value+'\\'+datasets.value
    script_dir = Path.cwd()

    abs_file_path = os.path.join(script_dir, rel_path)


    print(abs_file_path)


    if datasets.value.find('.csv') > -1:
        wsheets.description = 'Separator'
        wsheets.options = [',',';']

    if datasets.value.find('.tsv') > -1:
        wsheets.description = 'Separator'
        wsheets.options = ['\\t']

    if datasets.value.find('.xlsx') > -1:

        wsheets.description = 'Worksheets'
        xls = pd.ExcelFile(abs_file_path)
        wsheets.options = xls.sheet_names

    wslay.display = 'block'
    wsheets.value = wsheets.options[0]
    wsheets.layout = wslay

    butlay.display = 'block'

    return

##############################################################################################

def read_dataset(b):

    global DFPage,DataFolder,datasets,original_df,curr_df,wsheets,datasetname,resultexp,reslay
    global FeatPage,ProcssPage,processtypes

    rel_path = DataFolder.value+'\\'+datasets.value

    abs_file_path = os.path.join(Path.cwd(), rel_path)

    print('Path in reading the file (Opening a selected file): ',abs_file_path)

    if abs_file_path.find('.csv') > -1:
        curr_df = pd.read_csv(abs_file_path, sep=wsheets.value)
    if abs_file_path.find('.xlsx') > -1:
        xls = pd.ExcelFile(abs_file_path)
        curr_df = pd.read_excel(xls,wsheets.value)
    if abs_file_path.find('.tsv') > -1:
        curr_df = pd.read_csv(abs_file_path, sep="\t")

    curr_df.convert_dtypes()

    rel_path= rel_path[rel_path.find('\\')+1:]
    datasetname = rel_path[:rel_path.find('.')]


    reslay.height = '100px'
    resultexp.value = ''
    resultexp.layout = reslay

    processtypes.value = processtypes.options[0]

    with FeatPage:
        clear_output()
    with ProcssPage:
        clear_output()

    with DFPage:
        clear_output()
        #####################################
        display.display(curr_df.info())
        display.display(curr_df.describe())
        display.display(curr_df)
        #####################################

    return
#########################################################################################################

def on_submit_func(sender):

    global DataFolder


    rel_path = DataFolder.value

    current_path = os.getcwd()

    print(current_path)


    abs_file_path = os.path.join(Path.cwd(), rel_path)

    print('Path in reading the file (checking files in folder): ',abs_file_path)

    dtsetnames = []
    for root, dirs, files in os.walk(abs_file_path):
        for file in files:

            if (file.find('.csv')>-1) or (file.find('.xlsx')>-1) or(file.find('.tsv')>-1):
                dtsetnames.append(file)


    datasets.options = dtsetnames
    #datasets.value = dtsetnames[0]
    return


dtsetnames = []
rowheight = 18
curr_df = pd.DataFrame()
clean_df = pd.DataFrame()
original_df = pd.DataFrame()
datasetname = ''


wslay =  widgets.Layout()
wslay.display = 'none'
wsheets = widgets.Dropdown( options=[], description='..', disabled=False, layout = wslay)

DataFolder=widgets.Text(description ='Folder name:',value = 'DataSets')
datasets = widgets.Dropdown(options=[], description='DataSets:',layout = Layout(width='50%'))

butlay = Layout(width='75px')
butlay.display = 'none'
readfile = widgets.Button(description="Read",layout = butlay)

DataFolder.on_submit(on_submit_func)
datasets.observe(FileClick,'value')
readfile.on_click(read_dataset)




filelay =  widgets.Layout(height = '60px',width='99%')
tablayout = widgets.Layout(height='500px')
fthboxlay = widgets.Layout(height='500px')

DFPage = widgets.Output(layout=Layout(height='250px',align_items='center',overflow="visible"))

tab_1 = VBox(children=[
    HBox(children = [DataFolder,datasets,wsheets,readfile],layout=filelay),
    HBox(children = [DFPage],layout = fthboxlay)
                    ],layout=tablayout)

##################################################################################################################
###########################################  TAB: Data Cleaning ##################################################
##################################################################################################################
ShowMode = True


def ActivateTab1():

    global wslay,ftlay,features
    global tablayout,tab_1,dtlay,fthboxlay,ShowMode
    global rowheight,dtlay,dtypes,missing,msslay,misshands,mhndslay,curr_df,featurevals

    ShowMode = False

    msslay.display = 'block'
    msslay.height ='30px'
    mhndslay.display = 'block'
    mhndslay.height ='30px'
    ftlay.display = 'block'
    ftlay.height ='30px'
    dtlay.display = 'block'
    dtlay.height ='30px'

    featurevals.options = []

    features.options = curr_df.columns
    dtypes.options =[str(colid)+'-'+str(curr_df[curr_df.columns[colid]].dtype) for colid in  range(len(curr_df.columns))]

    currhghttxt = int(ftlay.height[:ftlay.height.find('px')])
    ftlay.height = str(min(450,currhghttxt+len(curr_df.columns)*rowheight))+'px'
    features.layout = ftlay

    currhghttxt = int(dtlay.height[:dtlay.height.find('px')])
    dtlay.height = str(min(450,currhghttxt+len(curr_df.columns)*rowheight))+'px'
    dtypes.layout = dtlay


    missing.options = [str(colid)+'-'+str(curr_df[curr_df.columns[colid]].isnull().sum()) for colid in  range(len(curr_df.columns))]
    msslay.height = ftlay.height
    missing.layout = msslay

    misshands.options = [str(colid)+'-'+'Keep'+','+'Remove' for colid in  range(len(curr_df.columns))]
    mhndslay.height = ftlay.height
    misshands.layout = mhndslay

    ShowMode = True

    return
##########################################################################################################
def HandleMissingValues(b):

    global misshands,curr_df,resultexp,clean_df
    clean_df = curr_df.copy()
    optind = 0
    rowheight = 16

    inisize = len(curr_df)
    inicols = len(curr_df.columns)

    resultexp.value += 'Initial data: '+str(inicols)+', columns,  size '+str(inisize)+'\n'

    nrrows = 0

    for opt in misshands.options:
        curact = misshands.options[optind]
        check = curact[curact.find('-')+1:]
        check = check[:check.find(',')]

        if check != 'Drop':
            if curact[curact.find(',')+1:] == 'Remove':

                clean_df = clean_df.dropna(subset = [clean_df.columns[optind]])
            if curact[curact.find(',')+1:] == 'Mean':
                clean_df[clean_df.columns[optind]].fillna(clean_df[clean_df.columns[optind]].mean(), inplace=True)
            if curact[curact.find(',')+1:] == 'Median':
                clean_df[clean_df.columns[optind]].fillna((clean_df[clean_df.columns[optind]].median()), inplace=True)
        optind+=1

    # check dropped columns
    optind = 0
    dropped = 0
    for opt in misshands.options:
        curact = misshands.options[optind]
        curact = curact[curact.find('-')+1:]
        curact = curact[:curact.find(',')]

        if curact == 'Drop':
            resultexp.value += 'Column drop: '+clean_df.columns[optind]+'\n'
            nrrows+=1
            clean_df = clean_df.drop([clean_df.columns[optind-dropped]], axis=1)
            dropped+=1

        optind+=1

    curr_df = clean_df
    ActivateTab1()

    resultexp.value += 'Cleaned data: '+str(len(clean_df.columns))+', columns, size  '+str(len(clean_df))+'\n'




    return
######################################################################################################################
def feature_click(event):
    global featurename,features,dtypes,missing,misshands
    global HCPage,featurevals

    if not ShowMode:
        return



    featurevals.options = []
    featurename.value = 'Selected Feature: '+features.value
    ratiosum = 0
    optind = 0
    datasize = len(curr_df)
    for opt in features.options:
        if features.value == opt:
            dtypes.value = dtypes.options[optind]
            missing.value = missing.options[optind]
            misshands.value = misshands.options[optind]
            typecheck = dtypes.value
            checkissing = missing.value

            if (checkissing[checkissing.find('-')+1:] == '0') and len(curr_df[curr_df.columns[optind]].unique()) <= 250:

                featurevals.options = [x for x in curr_df[curr_df.columns[optind]].unique()]

                if (typecheck[typecheck.find('-')+1:] == 'float64') or (typecheck[typecheck.find('-')+1:] == 'int64'):
                    featurevals.options = sorted(featurevals.options, key=lambda x: x, reverse=False)
                    ratiosum =sum([len(curr_df[curr_df[curr_df.columns[optind]] == x])/datasize for x in featurevals.options])

                featurevals.options = [str(x)+' ('+str(round(100*len(curr_df[curr_df[curr_df.columns[optind]] == x])/datasize,3))+'%)' for x in featurevals.options]
                featurevals.value = featurevals.options[0]

            break
        optind+=1


    checkissing = missing.value
    typecheck = dtypes.value


    if checkissing[checkissing.find('-')+1:] == '0':
        with HCPage:
            clear_output()

            if (typecheck[typecheck.find('-')+1:] == 'float64') or (typecheck[typecheck.find('-')+1:] == 'int64'):
                sns.distplot(curr_df[curr_df.columns[optind]]).set_title('Histogram of feature '+curr_df.columns[optind])
                plt.show()
            if (typecheck[typecheck.find('-')+1:] == 'object') or (typecheck[typecheck.find('-')+1:] == 'string'):

                nrclasses = len(curr_df[curr_df.columns[optind]].unique())
                if nrclasses < 250:
                    g = sns.countplot(curr_df, x=curr_df.columns[optind])
                    g.set_xticklabels(g.get_xticklabels(),rotation= 45)
                    #sns.distplot(curr_df[curr_df.columns[optind]]).set_title('Histogram of feature '+curr_df.columns[optind])
                    plt.show()
                else:
                    display.display('Number of classes: ',nrclasses)

    return

#####################################################################################################################
def action_click(change):

    global misshands,missingacts,features,dtypes



    selectid = 0
    optind = 0
    firstpart=  ''


    for opt in features.options:

        if opt == features.value:
            checktype = dtypes.options[optind]
            if (change['new'] == 'Mean') or  (change['new'] == 'Median'):
                if checktype[checktype.find('-')+1:] != 'float64':
                    return
            selectid = optind
            firstpart = misshands.options[selectid][:misshands.options[selectid].find(',')]
            break
        optind+=1


    newopt = firstpart+','+change['new']

    misshands.options = [misshands.options[optind] if optind!= selectid else newopt for optind in range(len(misshands.options))]



    return

##################################################################################################################
def column_click(change):
    global misshands,colmnacts,features,dtypes,missingacts

    if colmnacts.value == colmnacts.options[0]:
        return

    selectid = 0



    optind = 0
    for opt in features.options:
        if opt == features.value:
            selectid = optind
            break
        optind+=1

    prevstr = misshands.options[selectid]


    newopt = str(selectid)+'-'+change['new']+','+prevstr[prevstr.find(',')+1:]

    misshands.options = [misshands.options[optind] if optind!= selectid else newopt for optind in range(len(misshands.options))]
    colmnacts.value = colmnacts.options[0]

    return
#################################################################################################################
def featval_click(change):
    global misshands,colmnacts,features,dtypes,missingacts,curr_df,featurevals,valchangeacts,resultexp,reslay

    selectid = 0


    optind = 0
    for opt in featurevals.options:
        if opt == featurevals.value:
            selectid = optind
            break
        optind+=1

    colname = features.value



    featureval =  featurevals.value

    featureval =  featureval[:featureval.find('(')-1]


    prevsize = len(curr_df)

    if valchangeacts.value == 'Remove':
        curr_df = curr_df.drop(curr_df[curr_df[colname] == featureval].index)


    resultexp.value += 'Feature: '+colname+' value '+featureval+ ' removed. Size: '+str(prevsize)+'->'+str(len(curr_df))+'\n'
    valchangeacts.value == valchangeacts.options[0]

    return
##################################################################################################################
def savecurrdata(change):

    global curr_df,DataFolder

    version = 0



    if datasetname.find('_clean') > 0:

        checkstr = datasetname[datasetname.find('_clean')+1:]


        version = int(checkstr[checkstr.find('_v')+2:])

        dsname = datasetname[:datasetname.find('_clean')]

        filename = DataFolder.value+'/'+dsname+'_clean_v'+str(version+1)+'.csv'
        curr_df.to_csv(filename, index=False)

    else:

        filename = DataFolder.value+'/'+datasetname+'_clean_v'+str(version)+'.csv'
        curr_df.to_csv(filename, index=False)

    return

ftlay =  widgets.Layout(width='15%')
ftlay.display = 'none'
features = widgets.Select(options=[],layout = ftlay)


dtlay =  widgets.Layout(width='15%')
dtlay.display = 'none'
dtypes = widgets.Select(options=[],layout = dtlay)

features.observe(feature_click, 'value')

msslay =  widgets.Layout(width='15%')
msslay.display = 'none'
missing = widgets.Select(options=[],layout = msslay)

mbutlay = Layout(width='250px',align_items='center')
misshandbutton = widgets.Button(description="Apply Cleaning",layout = mbutlay)
savelay = Layout(width='250px',align_items='center')
savebutton = widgets.Button(description="Save Data",layout = savelay)

misshandbutton.on_click(HandleMissingValues)



#df[feature].isnull()

datsetname = widgets.Label(value ='')
types = widgets.Label(value ='D.Types')
featurename = widgets.Label(value ='')

ftshboxlay = widgets.Layout(height='500px',width='95%',align_items='center')


colmnacts = widgets.Dropdown( options=['Select','Keep','Drop'], description='', disabled=False,layout = widgets.Layout(width='125px'))
colmnacts.observe(column_click,'value')



missingacts = widgets.Dropdown( options=['Select','Remove','Mean','Mode','Median'], description='', disabled=False,layout = widgets.Layout(width='125px'))
featurevals = widgets.Dropdown( options=[], description='', disabled=False,layout = widgets.Layout(width='125px'))

valchangeacts = widgets.Dropdown( options=['Select','Remove'], description='', disabled=False,layout = widgets.Layout(width='125px'))
ftvallay = Layout(width='100px')
ftvalchn = widgets.Button(description="Handle Value",layout = ftvallay)
ftvalchn.on_click(featval_click)

svtlay = Layout(width='150px')
svebtn = widgets.Button(description="Save Current Data",layout = svtlay)
svebtn.on_click(savecurrdata)


mhndslay =  widgets.Layout(width='15%')
mhndslay.display = 'none'
misshands = widgets.Select(options=[],layout = mhndslay)

missingacts.observe(action_click,'value')

hbox0 = VBox(children = [datsetname])

vbox0 = VBox(children = [widgets.Label(value ='Features'),features])
vbox1 = VBox(children = [widgets.Label(value ='D.Types'),dtypes])
vbox2 = VBox(children = [widgets.Label(value ='Miss.'),missing])
vbox3 = VBox(children = [
        widgets.Label(value ='Column use'),colmnacts,widgets.Label(value ='Missing handling'),missingacts
        ,widgets.Label(value ='Uniq. Values'),featurevals,valchangeacts,ftvalchn])
vbox4 = VBox(children = [widgets.Label(value ='Actions'),misshands])

missingacts = widgets.Dropdown( options=['Select','Remove','Mean','Mode','Median'], description='', disabled=False,layout = widgets.Layout(width='125px'))

reslay = widgets.Layout(height='50x', width="99%")
resultexp = widgets.Textarea(value='', placeholder='',description='',disabled=True,layout = reslay)


HCPage = widgets.Output(layout=Layout(height='250px',align_items='center',overflow="visible"))

misslabel  = widgets.Label(value ='Missing Value Handling')
mssllay2 =  widgets.Layout(width='100%',align_items='center')
misslabel2 = widgets.Label(value =' __________________________________________________')

tab_2 = HBox(children = [


                       VBox(children=[misslabel,misslabel2,

                       HBox(children = [vbox0,vbox1,vbox2,vbox3,vbox4] )

                         ,misshandbutton,resultexp,svebtn]

                            ,layout = ftshboxlay),HCPage])

tab_2.observe(ActivateTab1,'value')
##################################################################################################################
#########################################  TAB: Data Processing   ################################################
##################################################################################################################
from sklearn.utils import resample
##################################################################################
def StandardizeColumn(df,colname):

    colmean = df[colname].mean()

    df[colname] = (df[colname]- colmean)/df[colname].std()

    return df

def NormalizeColumn(df,colname):

    col_min = min(df[colname])
    col_max = max(df[colname])

    if col_max == col_min:
        return

    df[colname] = (df[colname]- col_min)/(col_max-col_min)

    return df

####################################################################################
def ActivateTab2():


    global ftlay2,features2,curr_df



    ftlay2.display = 'block'
    ftlay2.height ='30px'

    features2.options = curr_df.columns


    currhghttxt = int(ftlay2.height[:ftlay2.height.find('px')])
    ftlay2.height = str(min(450,currhghttxt+len(curr_df.columns)*rowheight))+'px'
    features2.layout = ftlay2

    return

###############
def featurepr_click(event):
    global featurename,features2,dtypes,missing,misshands
    global FeatPage,ProcssPage,featurevals,curr_df,scalingacts,processtypes

    if not ShowMode:
        return

    colname = features2.value


    with FeatPage:
        clear_output()

        if (curr_df[colname].dtype == 'float64') or (curr_df[colname].dtype == 'int64'):

            if processtypes.value == 'Scaling':
                sns.distplot(curr_df[colname]).set_title('Histogram of feature '+colname)

                plt.legend(['Mean '+str(round(curr_df[colname].mean(),2)),'Stdev '+str(round(curr_df[colname].std(),2))], bbox_to_anchor=(0.6, 0.6))

                plt.show()

            if processtypes.value == 'Outlier':
                #######################################################################################################

                quantiles = curr_df[colname].quantile([0.25,0.5,0.75])
                IQR = quantiles[0.75] - quantiles[0.25]
                boxplot_outlierLB =  quantiles[0.25]-1.5*IQR
                boxplot_outlierUB =  quantiles[0.75]+1.5*IQR

                sns.boxplot(curr_df[colname]).set_title('Box plot of '+colname+' (Boundaries '+
                                                        str(round(boxplot_outlierLB,2))
                                                        +','+str(round(boxplot_outlierUB,2))+')')
                plt.show()


                curr_df["outlier"] = ((curr_df[colname])>boxplot_outlierUB) | ((curr_df[colname]<boxplot_outlierLB))

                outliers_df = curr_df[curr_df["outlier"] == True]

                display.display('Outlier raws: ')
                display.display(outliers_df)

                ############################################################################################################
            if processtypes.value == 'Imbalancedness':
                if len(curr_df[colname].unique()) == 2: # binary detection

                    plt.figure(figsize=(6, 2))
                    ax = sns.countplot(x=colname,data=curr_df, palette="cool_r")
                    for p in ax.patches:
                        ax.annotate("{:.1f}".format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01))
                    plt.show()


        if (curr_df[colname].dtype == 'object') or (curr_df[colname].dtype== 'string'):


            nrclasses = len(curr_df[colname].unique())
            if nrclasses < 250:
                g = sns.countplot(curr_df, x=colname)
                g.set_xticklabels(g.get_xticklabels(),rotation= 45)

                    #sns.distplot(curr_df[curr_df.columns[optind]]).set_title('Histogram of feature '+curr_df.columns[optind])
                plt.show()
            else:
                display.display('Number of classes: ',nrclasses)

    with ProcssPage:
        clear_output()



    scalingacts.value = scalingacts.options[0]
    return
##############
def makescaling(event):

    global featurename,features2,dtypes,missing,misshands
    global ProcssPage,featurevals,curr_df,scalingacts

    colname = features2.value


    if scalingacts.value == 'Standardize':
        curr_df =  StandardizeColumn(curr_df,colname)

        with ProcssPage:
            clear_output()
            sns.distplot(curr_df[colname]).set_title('Histogram of standardized feature '+colname)
            plt.legend(['Mean '+str(round(curr_df[colname].mean(),2)),'Stdev '+str(round(curr_df[colname].std(),2))], bbox_to_anchor=(0.6, 0.6))

            #plt.text(20,20, 'Mean= '+str(round(curr_df[colname].mean(),2)), dict(size=10))
            #plt.text(25,25, 'Stdev= '+str(round(curr_df[colname].std(),2)), dict(size=10))
            plt.show()

    if scalingacts.value == 'Normalize':
        curr_df =  NormalizeColumn(curr_df,colname)

        with ProcssPage:
            clear_output()

            sns.distplot(curr_df[colname]).set_title('Histogram of normalized feature '+colname)
            plt.legend(['Mean '+str(round(curr_df[colname].mean(),4)),'Stdev '+str(round(curr_df[colname].std(),4))], bbox_to_anchor=(0.6, 0.6))

            #plt.text(20,20, 'Mean= '+str(round(curr_df[colname].mean(),2)), dict(size=10))
            #plt.text(25,25, 'Stdev= '+str(round(curr_df[colname].std(),2)), dict(size=10))
            plt.show()

    return
#################################################################################################################
def makebalanced(event):

    global ProcssPage,featurevals,curr_df,balncacts,features2

    colname = features2.value

    if balncacts.value == 'Upsample':

        if len(curr_df[colname].unique()) == 2: # binary detection

            colvals = curr_df[colname].unique()
            ColmFirst = curr_df[ curr_df[colname] == colvals[0]]
            ColmOther = curr_df[ curr_df[colname] == colvals[1]]

            if len(ColmFirst) < len(ColmOther):
                upsampled_First = resample(ColmFirst, replace=True, n_samples=len(ColmOther), random_state=27)
                curr_df = pd.concat([ColmOther, upsampled_First])
            else:
                upsampled_Other= resample(ColmOther, replace=True, n_samples=len(ColmFirst), random_state=27)
                curr_df = pd.concat([ColmFirst, upsampled_Other])

            with ProcssPage:

                clear_output()
                plt.figure(figsize=(6, 2))
                ax = sns.countplot(x=colname,data=curr_df, palette="cool_r")
                for p in ax.patches:
                    ax.annotate("{:.1f}".format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01))
                plt.show()

    return
####################################################################################################################
def SelectProcessType(change):

    global processtypes,sclblly,scalelbl,prctlay,scalingacts,imblncdlay,balncacts,imbllbllly,imbllbl
    global outrmvlay,outrmvbtn

    sclblly.display = 'none'
    sclblly.visibility = 'hidden'
    scalelbl.layout = sclblly

    outrmvlay.display = 'none'
    outrmvlay.visibility = 'hidden'
    outrmvbtn.layout = outrmvlay

    imbllbllly.display = 'none'
    imbllbllly.visibility = 'hidden'
    imbllbl.layout = imbllbllly

    prctlay.visibility = 'hidden'
    scalingacts.layout = prctlay

    imblncdlay.visibility = 'hidden'
    balncacts.layout = imblncdlay

    if processtypes.value == 'Scaling':
        sclblly.display = 'block'
        sclblly.visibility = 'visible'
        scalelbl.layout = sclblly
        prctlay.visibility = 'visible'
        scalingacts.layout = prctlay

    if processtypes.value == 'Imbalancedness':

        imbllbllly.display = 'block'
        imbllbllly.visibility = 'visible'
        imbllbl.layout = imbllbllly
        imblncdlay.visibility = 'visible'
        balncacts.layout = imblncdlay

    if processtypes.value == 'Outlier':
        outrmvlay.display = 'block'
        outrmvlay.visibility = 'visible'
        outrmvbtn.layout = outrmvlay




    return

##################################################################################
def removeoutliers(change):

    global curr_df

    curr_df = curr_df[curr_df["outlier"] == False]
    curr_df = curr_df.drop(["outlier"], axis=1)

    return

##################################################################################

FeatPage = widgets.Output(layout=Layout(align_items='center',overflow="visible"))
ProcssPage = widgets.Output(layout=Layout(align_items='center',overflow="visible"))

svprtlay = Layout(width='150px')
sveprbtn = widgets.Button(description="Save Processed Data",layout = svprtlay)
sveprbtn.on_click(savecurrdata)

sclblly = widgets.Layout()
scalelbl = widgets.Label(value ='Scaling Methods',layout = sclblly)
sclblly.visibility = 'hidden'
scalelbl.layout = sclblly

prctlay = widgets.Layout()
scalingacts = widgets.Dropdown( options=['Select','Standardize','Normalize'], description='', disabled=False,layout = prctlay)
scalingacts.observe(makescaling,'value')

prctlay.visibility = 'hidden'
scalingacts.layout = prctlay



imbllbllly = widgets.Layout()
imbllbl = widgets.Label(value ='Imbalanced handling',layout = imbllbllly)
imbllbllly.visibility = 'hidden'
imbllbl.layout = imbllbllly

imblncdlay = widgets.Layout()
balncacts = widgets.Dropdown( options=['Select','Upsample','DownSample'], description='', disabled=False,layout = prctlay)
balncacts.observe(makebalanced,'value')

imblncdlay.visibility = 'hidden'
balncacts.layout = imblncdlay


outrmvlay = Layout(width='150px')
outrmvbtn = widgets.Button(description="Remove Outliers",layout = outrmvlay)
outrmvbtn.on_click(removeoutliers)
outrmvlay.visibility = 'hidden'
outrmvbtn.layout = outrmvlay

processtypes = widgets.Dropdown( options=['Select','Scaling','Outlier','Imbalancedness'], description='', disabled=False)
processtypes.observe(SelectProcessType,'value')


ftlay2 =  widgets.Layout()
ftlay2.display = 'none'
features2 = widgets.Select(options=[],description = '',layout = ftlay2)
features2.observe(featurepr_click, 'value')

vbox1 = VBox(children = [widgets.Label(value ='Features'),features2,
                         widgets.Label(value ='Process Types'),processtypes,scalelbl,scalingacts,
                         imbllbl,balncacts,outrmvbtn,sveprbtn])
vbox2 = VBox(children = [FeatPage,ProcssPage])



tab_3 = HBox(children=[vbox1,vbox2])





tab_set = widgets.Tab([tab_1,tab_2,tab_3])
tab_set.set_title(0, 'Data Sets')
tab_set.set_title(1, 'Data Cleaning')
tab_set.set_title(2, 'Data Processing')


tab_set.observe(ActivateTab,'selected_index')

tab_set

/content
Path in reading the file (checking files in folder):  /content/DataSets
/content
Path in reading the file (checking files in folder):  /content/DataSets


----------
## PART 2: Data Aggragation and Integration

_Data set:_ World bank data

This data set is in horizontal form. The data points are countries and the columns represent the indicator value of the countries for a specific year.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


def ReadFile(url,sepatr):
    df = pd.read_csv(url, sep=sepatr)
    df = df.convert_dtypes()
    return df

def SumColumns(x,col1,col2):
    return x[col1]*x[col2]


print('>> Reading a vertical data: key information is included in rows by repeating some other column values...')

url ="https://raw.githubusercontent.com/bprasad26/lwd/master/data/gapminder.tsv"
sepatr = '\t'
df_wp = ReadFile(url,sepatr)

df_wp.head(20)

# Note that in year column year values have increment value (granularity) 5

>> Reading a vertical data: key information is included in rows by repeating some other column values...


,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.85303
2,Afghanistan,Asia,1962,31.997,10267083,853.10071
3,Afghanistan,Asia,1967,34.02,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106
5,Afghanistan,Asia,1977,38.438,14880372,786.11336
6,Afghanistan,Asia,1982,39.854,12881816,978.011439
7,Afghanistan,Asia,1987,40.822,13867957,852.395945
8,Afghanistan,Asia,1992,41.674,16317921,649.341395
9,Afghanistan,Asia,1997,41.763,22227415,635.341351


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

print('>> Reading a horizontal data: Key information is included in columns..')

url = "https://github.com/muratfirat78/CPP_Datasets/raw/main/API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_4481203.csv"
df_wb = ReadFile(url,',')
# drop unnnecessary columns
df_wb = df_wb.drop(["1960","1961","Unnamed: 66","Indicator Code","Indicator Name"], axis=1)

df_wb[['Country Name']+[str(y) for y in range(1962,2009)]].head(10)

# Note that years of columns have increment value (granularity) 1


>> Reading a horizontal data: Key information is included in columns..


,Country Name,1962,1963,1964,1965,1966,1967,1968,1969,1970,...,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008
0,Aruba,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,2.280372,4.044021,2.883604,3.315247,3.656365,2.529129,3.397787,3.608024,5.392568,8.955987
1,Africa Eastern and Southern,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,7.819865,8.601485,5.840354,8.763754,7.4497,5.023421,8.558038,8.898164,8.450775,12.566645
2,Afghanistan,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,12.686269,6.784597,8.680571,26.418664
3,Africa Western and Central,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.372266,2.530775,4.361529,3.188693,1.760904,0.694336,5.631634,4.4159,3.607368,8.449722
4,Angola,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,248.195902,324.996872,152.561022,108.897436,98.224144,43.542107,22.953514,13.30521,12.251497,12.475829
5,Albania,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.389438,0.050018,3.107588,7.770526,0.484003,2.280019,2.366582,2.370728,2.932682,3.320871
6,Andorra,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,Arab World,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,2.669374,1.85379,1.772204,1.832994,2.712592,3.63228,3.493685,3.544986,4.743906,11.270665
8,United Arab Emirates,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,12.25042
9,Argentina,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


----------
_Data set:_ World bank data

df_wbvert is the vertical dataframe in which we have inflation values of countries for every 5-year periods. The inflation values of this dataframe is filled in FillInflation function. Note that df_wbvert dataframe is initialized in a aligned way of the horizontal dataframe df_wb. Go through the function FillInflation to see how values are assigned from one dataframe to another.


In [ ]:
######################################################################################################
year_range = range(1962,2008,1)
print('>> Defining a vertical data from horizontal data')
df_wbvert = pd.DataFrame(columns=[])
df_wbvert["country"] = [country for country in (df_wb["Country Name"]).tolist() for x in year_range ]
df_wbvert["year"] = [x for country in df_wb["Country Name"].tolist() for x in year_range ]
df_wbvert["inflation"] = np.zeros(shape=(len(df_wbvert),1)) # inflation is initialized as zero

# inflation values are filled for 5-year intervals
years = []
for year in year_range:
    years.append(str(year))
    if (year - 1962)%5 ==0:
        lstval = [x for x in df_wb[years].mean(axis = 1,skipna = True)]
        df_wbvert["inflation"][df_wbvert["year"] == year]=lstval
        years.clear()

df_wbvert[df_wbvert['year'] == 2007].head(20)
######################################################################################################

>> Defining a vertical data from horizontal data


,country,year,inflation
45,Aruba,2007,3.716775
91,Africa Eastern and Southern,2007,7.676019
137,Afghanistan,2007,9.383812
183,Africa Western and Central,2007,3.222028
229,Angola,2007,38.055294
275,Albania,2007,2.086803
321,Andorra,2007,<NA>
367,Arab World,2007,3.62549
413,United Arab Emirates,2007,<NA>
459,Argentina,2007,<NA>


In [ ]:
############################################################################################################
print('Merging two vertical data sets: Left (base) data set has size', len(df_wp),' right ',len(df_wbvert))
df_merged = pd.merge(df_wp, df_wbvert,  how="left", on=["country","year"])
print('Size of merged data set: ',len(df_merged))
df_merged[df_merged["inflation"].notnull()].head(10)
############################################################################################################

Merging two vertical data sets: Left (base) data set has size 1704  right  12236
Size of merged data set:  1704


,country,continent,year,lifeExp,pop,gdpPercap,inflation
2,Afghanistan,Asia,1962,31.997,10267083,853.10071,2.05255
3,Afghanistan,Asia,1967,34.02,11537966,836.197138,2.923182
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,4.761923
6,Afghanistan,Asia,1982,39.854,12881816,978.011439,17.149287
7,Afghanistan,Asia,1987,40.822,13867957,852.395945,17.365561
9,Afghanistan,Asia,1997,41.763,22227415,635.341351,1.967976
10,Afghanistan,Asia,2002,42.129,25268405,726.734055,-3.030738
16,Albania,Europe,1972,67.69,2263554,3313.422188,4.141556
18,Albania,Europe,1982,70.42,2780097,3630.880722,45.581259
20,Albania,Europe,1992,71.581,3326498,2497.437901,7.484167


In [ ]:
right = pd.DataFrame(
   {
      "key1": ["K0", "K1", "K1", "K2"],
      "key2": ["K0", "K0", "K0", "K0"],
      "C": ["C0", "C1", "C2", "C3"],
      "D": ["D0", "D1", "D2", "D3"],
   }
)

right

,key1,key2,C,D
0,K0,K0,C0,D0
1,K1,K0,C1,D1
2,K1,K0,C2,D2
3,K2,K0,C3,D3


In [ ]:
left = pd.DataFrame(
   {
      "key1": ["K0", "K0", "K1", "K2"],
      "key2": ["K0", "K1", "K0", "K1"],
      "A": ["A0", "A1", "A2", "A3"],
      "B": ["B0", "B1", "B2", "B3"],
   }
)
left

,key1,key2,A,B
0,K0,K0,A0,B0
1,K0,K1,A1,B1
2,K1,K0,A2,B2
3,K2,K1,A3,B3


In [ ]:
###############################################################
result = pd.merge(left, right, how="inner", on=["key1", "key2"])
###############################################################

result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


----------
### Data Aggragation


In [ ]:
import pandas as pd
data = {'item' :['tea','coffee','sugar','tea','coffee','sugar','tea','coffee','sugar'],
        'year':[2010,2010,2010,2011,2011,2011,2012,2012,2012],
        'sales':[1200,1050,500,1500,1200,1000,1230,1300,1420]}
df = pd.DataFrame(data,columns=['item','year','sales'])
df

,item,year,sales
0,tea,2010,1200
1,coffee,2010,1050
2,sugar,2010,500
3,tea,2011,1500
4,coffee,2011,1200
5,sugar,2011,1000
6,tea,2012,1230
7,coffee,2012,1300
8,sugar,2012,1420


In [ ]:
###################################################
item_group = df.groupby('item')
item_group.groups
###################################################


{'coffee': [1, 4, 7], 'sugar': [2, 5, 8], 'tea': [0, 3, 6]}

In [ ]:
###################################################
agg_group = df.groupby('item')['sales'].mean().reset_index()
agg_group
###################################################

,item,sales
0,coffee,1183.333333
1,sugar,973.333333
2,tea,1310.000000


In [ ]:
#############################################################################
item_group = df.groupby('item')['sales'].agg(lambda x:list(x)).reset_index()
item_group
#############################################################################

,item,sales
0,coffee,"[1050, 1200, 1300]"
1,sugar,"[500, 1000, 1420]"
2,tea,"[1200, 1500, 1230]"


In [ ]:
#############################################################################
item_group = df.groupby('item')['sales'].agg(lambda x:set(x)).reset_index()
item_group
#############################################################################

,item,sales
0,coffee,"{1200, 1050, 1300}"
1,sugar,"{1000, 1420, 500}"
2,tea,"{1200, 1500, 1230}"
